In [1]:
import tweepy
import json
from time import sleep
import pickle
import re
from os import listdir
from os.path import isfile, join
import sys

# configuration

In [31]:
config = {}
path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\CS597\\twitterMining\\'
initialSet_path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\InitialSet\\Football\\location\\'

config['KEYS'] = {'consumer_key': ['mVJcyNN5qFyk7GdBJvENVc329', 'anqjETSjwaDqA9LBt0MGwub60','vDnzvZFFwXFgI2ftQxYWV2XZo',
                                  'sAkNaJxJWgPilhidaNJ48kmCo','tO2Bzbkexyf57LSF6aJwACXfs'],
                  'consumer_secret': ['DbXUXB7EN77HmzQahobHeTPSWgcNI9t8kKzDoe5twfTORnjXxf',\
                                       'AGTAiJhOrb2gPSbW1F5Rclu0Ns7pTZthPhJ3i7ySFNWwdheqim',\
                                     '5HxLPob3kGqgauc5HuNszFw0uoLjsK3JHD0IqUNEVbNAIVdRKQ',\
                                      'dPcVmoYaEGhsWC5djrEFsjfNdO8DmFecmSnBRjL63y0Sg9pCtl',
                                     'OIEIE1Pu01eTieDffG13kJM78spAPHZtKk2IZ1yGkU0eoKvVxn'],
                  'access_token': ['707770026573561856-Vfqm0iXrsBvFL2prQMQ2Gzu9eawWYIj',\
                                    '707770026573561856-diXgIsR0UCYM3CHYCUoU38hcp8Qk8Pj',\
                                  '707770026573561856-9BcLFzjLrkVdxeG0Z3dmqYhsCDqUQ2t',\
                                  '707770026573561856-Sua2gUtlWTxTik70a0dUba0SvayeQgU',\
                                  '707770026573561856-Oi1nNb4Ih0SJyizIwToY3PhjLymGkcl'],
                  'access_token_secret': ['HSMqS4YkDBcB6mm9JL3vd6VeKlkKmNdMxmJ0inJQwNhXS',\
                                           'GnqrY0AoeOuK7cgaF2yRBmcJGcc814bBIONMREN8gU7xu',\
                                         'e2OfQnQGnweLygFxJRFCUa9PZhOZXyCcaEipMpnlmABqj',\
                                         'Stj2TQtwsrTQFkdronpZDUyrl9g3slZjEMxgIYznbmSzy',\
                                         'Cp3d4xKdTfPyJq6y7uLeRLv77kTjZmEIpGYtiWrrqvK6i']
                 }
''
# later add {if it's not exist then write, otherwise skip}
with open(path + "CONFIG_PATH.pickle", 'wb+') as f:
        pickle.dump(config, f)

In [32]:
consumer_key = config['KEYS']['consumer_key']    # list
consumer_secret = config['KEYS']['consumer_secret']
access_token = config['KEYS']['access_token']
access_token_secret = config['KEYS']['access_token_secret']

# set stream

In [40]:
class MyStreamListener(tweepy.StreamListener):

    def __init__(self, api = None):
        self.api = api or API()
        self.counter = 0
        self.info = []

    def on_data(self, data):
       
        if data != None:
            #status = json.loads(data)
            #print(data)
            self.on_status(data)
        else:
            #print('not ok')
            return 
    
    def on_error(self, status):
        print(status)

    def on_status(self, data):
        #print(status)
        status = json.loads(data)
        if status['user']['location'] != None: 
            if status['user']['location'].lower().find('chicago') != -1:
                print(status['user']['location'])

                self.info.append(status)
                self.counter += 1

                if self.counter >= 10:
                    with open(initialSet_path + 'stream'+str(self.counter)+'.json','w') as f:
                        json.dump(self.info,f)
                    sys.exit()
            
        return

    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...\n")
        time.sleep(60)
        return 

# get initial set

In [5]:
auth = tweepy.OAuthHandler(consumer_key[0], consumer_secret[0])
auth.set_access_token(access_token[0], access_token_secret[0])

In [6]:
def collect_tweets(topic):
    
    stream = tweepy.Stream(auth, MyStreamListener(api))
    stream.filter(track=topic, languages = ['en'])   
        
    return None

# test block 
print out the user tweets and location 

In [7]:
def test_block(path):
    
    filenames = [f for f in listdir(path) if isfile(join(path, f))]
    football_tw = []
    for filename in filenames:    
        
        with open(path + filename,'r') as f:
            football_tw += json.load(f)
    i = 0
    for item in football_tw:
        print('list id:', i)   # in order to mannually delete this irrelevant tweets
        print('userID:',item['user']['screen_name'])
        print('text:', item['text'])
        print('location:', item['user']['location'])
        print('\n')
        i += 1
   
    return football_tw

In [8]:
with open(initialSet_path + 'initial_list.pickle','rb') as f:
        initial_list = pickle.load(f)

In [9]:
len(initial_list)

86

# query API to find their representative list

In [10]:
auth = tweepy.OAuthHandler(consumer_key[0], consumer_secret[0])
auth.set_access_token(access_token[0], access_token_secret[0])
api = tweepy.API(auth, wait_on_rate_limit= True,wait_on_rate_limit_notify=True)

In [11]:
rep_path = initialSet_path+'\\rep\\'

# get the detail of a id

In [12]:
def get_screen_name(id_list):
    
    info = []
    for id in id_list:
        info.append(api.get_user(id))
        
    return info

# find the complete followers of the representative nodes

In [5]:
with open(initialSet_path + 'represent_id.pickle','rb') as f:
    represent_id = pickle.load(f)

In [6]:
ff_path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\InitialSet\\Football\\location\\ff_of_rep\\'

In [23]:
auth = tweepy.OAuthHandler(consumer_key[0], consumer_secret[0])
auth.set_access_token(access_token[0], access_token_secret[0])
api = tweepy.API(auth, wait_on_rate_limit= True,wait_on_rate_limit_notify=True)

In [69]:
auth = tweepy.OAuthHandler(consumer_key[1], consumer_secret[1])
auth.set_access_token(access_token[1], access_token_secret[1])
api = tweepy.API(auth, wait_on_rate_limit= True,wait_on_rate_limit_notify=True)

In [96]:
auth = tweepy.OAuthHandler(consumer_key[2], consumer_secret[2])
auth.set_access_token(access_token[2], access_token_secret[2])
api = tweepy.API(auth, wait_on_rate_limit= True,wait_on_rate_limit_notify=True)

In [122]:
auth = tweepy.OAuthHandler(consumer_key[3], consumer_secret[3])
auth.set_access_token(access_token[3], access_token_secret[3])
api = tweepy.API(auth, wait_on_rate_limit= True,wait_on_rate_limit_notify=True)

In [33]:
auth = tweepy.OAuthHandler(consumer_key[4], consumer_secret[4])
auth.set_access_token(access_token[4], access_token_secret[4])
api = tweepy.API(auth, wait_on_rate_limit= True,wait_on_rate_limit_notify=True)

In [34]:
for ID in represent_id[90:100]:
    with open(ff_path + str(ID) +'.pickle','wb+') as f:
        pickle.dump(api.followers_ids(str(ID)),f)

# test whether the ranking predictive model works

<p>seperate them into groups: baseline(twitter stream), every 300? </p>
<p>we should see there is a decline in matching accuracy in the two aspects: eg.football + chicago </p>

<p> 1. chicago </p>
<b> ratio = # user from chicago / # number of user provide their location </b>



In [132]:
with open(initialSet_path + 'ranked_candidate_id.pickle','rb') as f:
    candidate_id = pickle.load(f)
candidate_id[:10]

[(781149191808712704, -1758.2236764613422),
 (791068282736484352, -1762.922426437889),
 (785599640146108417, -1842.8592383852822),
 (779659530242142208, -1844.5119806159519),
 (787488684912156672, -1867.8320771088486),
 (754013832108994560, -1871.9273272390972),
 (780206098431213568, -1872.5019793483407),
 (550557628, -1876.569656086811),
 (788347210698256384, -1882.9656627997808),
 (790322349098749952, -1890.5031798600978)]

In [133]:
def gather_location(candidate_id, start, end):
    
    location_list = []
    for c_id in candidate_id[start:end]:
        try:
            location = api.get_user(str(c_id[0])).location
            if location == '':
                location_list.append('NaN')
            else:
                location_list.append(location)
        except:
            pass
            
    return location_list

In [134]:
location_300 = gather_location(candidate_id, start = 0, end = 300)
location_600 = gather_location(candidate_id, start = 300, end = 600)

In [139]:
location_900 = gather_location(candidate_id, start = 600, end = 900)

In [125]:
location_50000 = gather_location(candidate_id, start = 7000, end = 7528)

Rate limit reached. Sleeping for: 599


In [135]:
def print_ratio(location_list):
    
    target_count = 0
    general_count = 0

    for loc in location_list:

        #loc.lower().find('IL') != -1 or 
        if loc.lower().find('chicago') != -1:
            target_count += 1
            #print(loc)
        elif loc != 'NaN':
            general_count += 1

    ratio = target_count / general_count
    print("# of people from chicago : ", target_count)
    print("# of people provide location : ", general_count)
    print("# of people from chicago/ # of people provide location :", ratio)

    return ratio  

In [136]:
ratio_300 = print_ratio(location_300)

# of people from chicago :  48
# of people provide location :  119
# of people from chicago/ # of people provide location : 0.40336134453781514


In [138]:
ratio_600 = print_ratio(location_600)

# of people from chicago :  29
# of people provide location :  101
# of people from chicago/ # of people provide location : 0.2871287128712871


In [118]:
ratio_900 = print_ratio(location_900)

# of people from chicago :  25
# of people provide location :  145
# of people from chicago/ # of people provide location : 0.1724137931034483


In [127]:
ratio_50000 = print_ratio(location_50000)

# of people from chicago :  45
# of people provide location :  257
# of people from chicago/ # of people provide location : 0.17509727626459143


In [120]:
len(candidate_id)

7529

In [65]:
# set stream
rand_path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\RandSet\\'

class MyStreamListener(tweepy.StreamListener):

    def __init__(self, api = None):
        self.api = api or API()
        self.counter = 0
        self.info = []

    def on_data(self, data):
       
        if data != None:
            #status = json.loads(data)
            #print(data)
            self.on_status(data)
        else:
            #print('not ok')
            return 
    
    def on_error(self, status):
        print(status)

    def on_status(self, data):
        #print(status)
        status = json.loads(data)
        
        # those user who provide location information
        if status['user']['location'] != None: 
            #print(status['user']['location'])
            self.info.append(status)
            self.counter += 1
            
            if self.counter >= 100:
                with open(rand_path + 'stream'+str(self.counter)+'.json','w') as f:
                    json.dump(self.info,f)
                sys.exit()
            
        return

    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...\n")
        time.sleep(60)
        return 

In [47]:
auth = tweepy.OAuthHandler(consumer_key[0], consumer_secret[0])
auth.set_access_token(access_token[0], access_token_secret[0])
api = tweepy.API(auth, wait_on_rate_limit= True,wait_on_rate_limit_notify=True)

In [57]:
def rand_tweets():
    
    stream = tweepy.Stream(auth, MyStreamListener(api))
    stream.filter(track=["a", "the", "i", "you","it","of"], languages = ['en'])   
        
    return None

In [61]:
collect_tweets(['a','the','of'])

420


KeyboardInterrupt: 

# random test area

In [39]:
#api.get_user('786326788246999040')